In [1]:
import json
import pandas
import numpy as np
from scipy.interpolate import interp1d

In [2]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
data_directory = '/content/gdrive/MyDrive/bigData/datasets/data_Dota.json'

In [4]:
%%time
file_json = json.load(open(data_directory, 'r'))
matches = file_json['matches']

CPU times: user 34.1 ms, sys: 25.1 ms, total: 59.2 ms
Wall time: 71.4 ms


In [5]:
%%time
max_series_size = max([len(match['radiant_gold_adv']) for match in matches])
print(max([len(match['radiant_xp_adv']) for match in matches]))

80
CPU times: user 1.25 ms, sys: 72 µs, total: 1.33 ms
Wall time: 1.27 ms


In [132]:
max_series_size = 40

In [127]:
from statistics import median

In [129]:
max_series_size = median([len(match['radiant_gold_adv']) for match in matches])
print(max_series_size)

79.0


In [133]:
%%time
for i in range(len(matches)):
  # For each time series we create a formula, and we generate a certain amount of values using that.
  timestamps = [j * 60 for j in range(0, len(matches[i]['radiant_gold_adv']))]

  if len(matches[i]['radiant_gold_adv']) != len(matches[i]['radiant_xp_adv']):
    print(len(timestamps), timestamps)
    print(len(matches[i]['radiant_gold_adv']), matches[i]['radiant_gold_adv'])
    print(len(matches[i]['radiant_xp_adv']), matches[i]['radiant_xp_adv'])

  # We need at least two values to interpolate.
  if len(timestamps) > 1:
    interpolated_gold_function = interp1d(timestamps, matches[i]['radiant_gold_adv'])
    interpolated_xp_function = interp1d(timestamps, matches[i]['radiant_xp_adv'])

    x = np.linspace(timestamps[0], timestamps[-1], max_series_size)

    matches[i]['radiant_gold_adv'] = [interpolated_gold_function(j) for j in x]
    matches[i]['radiant_xp_adv'] = [interpolated_xp_function(j) for j in x]


CPU times: user 16.8 s, sys: 1.06 s, total: 17.9 s
Wall time: 17.3 s


In [10]:
%%time
data = pandas.DataFrame(matches)
data = data[data['duration'] >= 600]
data[['radiant_gold_adv_at_' + str(i) for i in range(max_series_size)]] = pandas.DataFrame(data.radiant_gold_adv.tolist(), index = data.index)
data[['radiant_xp_adv_at_' + str(i) for i in range(max_series_size)]] = pandas.DataFrame(data.radiant_xp_adv.tolist(), index = data.index)

data.drop('radiant_gold_adv', axis=1, inplace=True)
data.drop('radiant_xp_adv', axis=1, inplace=True)

CPU times: user 400 ms, sys: 21.5 ms, total: 422 ms
Wall time: 430 ms


In [ ]:
data

,match_id,duration,radiant_win,radiant_gold_adv_at_0,radiant_gold_adv_at_1,radiant_gold_adv_at_2,radiant_gold_adv_at_3,radiant_gold_adv_at_4,radiant_gold_adv_at_5,radiant_gold_adv_at_6,radiant_gold_adv_at_7,radiant_gold_adv_at_8,radiant_gold_adv_at_9,radiant_gold_adv_at_10,radiant_gold_adv_at_11,radiant_gold_adv_at_12,radiant_gold_adv_at_13,radiant_gold_adv_at_14,radiant_gold_adv_at_15,radiant_gold_adv_at_16,radiant_gold_adv_at_17,radiant_gold_adv_at_18,radiant_gold_adv_at_19,radiant_gold_adv_at_20,radiant_gold_adv_at_21,radiant_gold_adv_at_22,radiant_gold_adv_at_23,radiant_gold_adv_at_24,radiant_gold_adv_at_25,radiant_gold_adv_at_26,radiant_gold_adv_at_27,radiant_gold_adv_at_28,radiant_gold_adv_at_29,radiant_gold_adv_at_30,radiant_gold_adv_at_31,radiant_gold_adv_at_32,radiant_gold_adv_at_33,radiant_gold_adv_at_34,radiant_gold_adv_at_35,radiant_gold_adv_at_36,...,radiant_xp_adv_at_40,radiant_xp_adv_at_41,radiant_xp_adv_at_42,radiant_xp_adv_at_43,radiant_xp_adv_at_44,radiant_xp_adv_at_45,radiant_xp_adv_at_46,radiant_xp_adv_at_47,radiant_xp_adv_at_48,radiant_xp_adv_at_49,radiant_xp_adv_at_50,radiant_xp_adv_at_51,radiant_xp_adv_at_52,radiant_xp_adv_at_53,radiant_xp_adv_at_54,radiant_xp_adv_at_55,radiant_xp_adv_at_56,radiant_xp_adv_at_57,radiant_xp_adv_at_58,radiant_xp_adv_at_59,radiant_xp_adv_at_60,radiant_xp_adv_at_61,radiant_xp_adv_at_62,radiant_xp_adv_at_63,radiant_xp_adv_at_64,radiant_xp_adv_at_65,radiant_xp_adv_at_66,radiant_xp_adv_at_67,radiant_xp_adv_at_68,radiant_xp_adv_at_69,radiant_xp_adv_at_70,radiant_xp_adv_at_71,radiant_xp_adv_at_72,radiant_xp_adv_at_73,radiant_xp_adv_at_74,radiant_xp_adv_at_75,radiant_xp_adv_at_76,radiant_xp_adv_at_77,radiant_xp_adv_at_78,radiant_xp_adv_at_79
2,6292239339,1202,True,0.0,16.70886075949367,33.41772151898734,50.12658227848102,55.46835443037979,-155.1645569620252,-365.7974683544304,-576.4303797468355,-755.9746835443038,-655.7215189873418,-555.4683544303798,-455.21518987341784,-342.5822784810126,-159.79746835443038,22.987341772151865,205.77215189873402,322.1265822784811,172.75949367088623,23.392405063291335,-125.97468354430356,-278.12658227848067,-438.6329113924047,-599.1392405063286,-759.6455696202527,-906.0253164556963,-1019.4430379746836,-1132.860759493671,-1246.2784810126582,-1282.0759493670887,-1173.7215189873418,-1065.367088607595,-957.0126582278483,-863.848101265823,-793.4683544303799,-723.0886075949369,-652.7088607594939,-552.9367088607598,...,6945.949367088608,7017.848101265822,7089.7468354430375,7161.645569620253,7230.202531645569,7296.025316455696,7361.848101265822,7427.670886075949,8107.316455696204,9196.177215189875,10285.037974683546,11373.898734177215,12527.26582278481,13715.367088607594,14903.468354430379,16091.569620253164,17196.20253164557,18265.06329113924,19333.924050632908,20402.78481012658,23044.36708860759,26210.189873417716,29376.01265822784,32541.835443037962,32221.88607594937,31030.49367088608,29839.10126582279,28647.7088607595,28438.443037974685,28402.493670886077,28366.54430379747,28330.59493670886,33434.443037974655,39109.37974683545,44784.31645569617,50459.25316455697,56051.69620253161,61639.79746835443,67227.89873417717,72816.0
3,6292239088,1275,False,0.0,-3.721518987341772,-7.443037974683544,-11.164556962025316,18.27848101265822,153.84810126582283,289.4177215189873,424.9873417721518,346.6329113924051,32.962025316455595,-280.708860759494,-594.3797468354428,-911.8481012658227,-1230.8354430379748,-1549.8227848101262,-1868.8101265822784,-2015.6455696202531,-2153.8734177215188,-2292.1012658227846,-2411.3924050632913,-2450.2025316455697,-2489.012658227848,-2527.8227848101264,-2831.164556962026,-3487.2151898734173,-4143.265822784809,-4799.316455696203,-5094.911392405063,-5210.278481012658,-5325.645569620253,-5441.012658227848,-5295.670886075949,-5122.886075949367,-4950.1012658227855,-4947.594936708858,-5966.759493670888,-6985.924050632912,...,-2515.9367088607573,-1715.8101265822775,-2146.8987341772154,-3335.6582278481,-4524.417721518985,-5713.177215189869,-5106.49

In [ ]:
data.to_csv('dataInterpolada80.csv')
files.download('dataInterpolada80.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Preparation with Ray (Comparison)

In [6]:
!pip install ray
import ray

In [22]:
num_workers = 4
trials = 20

In [8]:
ray.init(num_cpus=num_workers, ignore_reinit_error=True)

{'metrics_export_port': 61621,
 'node_id': '2d07cb689c4ec0d2604c0955b6510f51f95135087541793abb9f1a4a',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-11-28_20-19-11_009018_1973/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-11-28_20-19-11_009018_1973/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-11-28_20-19-11_009018_1973',
 'webui_url': None}

In [13]:
@ray.remote
def load_data():
  file_json = json.load(open(data_directory, 'r'))
  return file_json['matches']

@ray.remote
def calc_max_size(matches):
  return max([len(match['radiant_gold_adv']) for match in matches])

In [26]:
@ray.remote
class DotaData(object):
  def __init__(self, matches, max_series_size):
    self.matches = matches
    self.max_series_size = max_series_size 

  def interpolate(self):
    for i in range(len(self.matches)):
      # For each time series we create a formula, and we generate a certain amount of values using that.
      timestamps = [j * 60 for j in range(0, len(self.matches[i]['radiant_gold_adv']))]

      # We need at least two values to interpolate.
      if len(timestamps) > 1:
        interpolated_gold_function = interp1d(timestamps, self.matches[i]['radiant_gold_adv'])
        interpolated_xp_function = interp1d(timestamps, self.matches[i]['radiant_xp_adv'])

        x = np.linspace(timestamps[0], timestamps[-1], self.max_series_size)

        self.matches[i]['radiant_gold_adv'] = [interpolated_gold_function(j) for j in x]
        self.matches[i]['radiant_xp_adv'] = [interpolated_xp_function(j) for j in x]

  def get_matches(self):
    return self.matches

In [14]:
@ray.remote
def create_dataframe():
  data = pandas.DataFrame(matches)
  data = data[data['duration'] >= 600]
  data[['radiant_gold_adv_at_' + str(i) for i in range(max_series_size)]] = pandas.DataFrame(data.radiant_gold_adv.tolist(), index = data.index)
  data[['radiant_xp_adv_at_' + str(i) for i in range(max_series_size)]] = pandas.DataFrame(data.radiant_xp_adv.tolist(), index = data.index)

  data.drop('radiant_gold_adv', axis=1, inplace=True)
  data.drop('radiant_xp_adv', axis=1, inplace=True)

  return data

In [15]:
@ray.remote
def normalize():
  normalized_data = data.copy()
  normalized_data.iloc[:,3:] = StandardScaler().fit_transform(normalized_data.iloc[:,3:])

  normalized_data_target = normalized_data['radiant_win']
  normalized_data.drop('radiant_win', axis=1, inplace=True)

  return normalized_data, normalized_data_target

In [16]:
%%time
ref = load_data.remote()
matches = ray.get(ref)

CPU times: user 85.7 ms, sys: 27.3 ms, total: 113 ms
Wall time: 2.05 s


In [17]:
%%time
ref = calc_max_size.remote(matches)
max_series_size = ray.get(ref)

CPU times: user 82.8 ms, sys: 12.4 ms, total: 95.2 ms
Wall time: 1.84 s


In [27]:
%%time 
dota_data = DotaData.remote(matches, max_series_size)
dota_data.interpolate.remote()

CPU times: user 24 ms, sys: 2.3 ms, total: 26.3 ms
Wall time: 31 ms


In [28]:
ref = dota_data.get_matches.remote()

In [102]:
%%time
ref = create_dataframe.remote()
dota_dataframe = ray.get(ref)

CPU times: user 3.31 s, sys: 175 ms, total: 3.48 s
Wall time: 8.61 s


In [106]:
%%time 
ref = normalize.remote()
normalized_data, normalized_data_target = ray.get(ref)

2021-11-28 19:33:56,564	WARNING worker.py:1228 -- Warning: The remote function __main__.normalize is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


CPU times: user 5.86 s, sys: 296 ms, total: 6.16 s
Wall time: 9.33 s


# Here PySpark begins~

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

--2021-11-28 04:27:21--  https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220400553 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.3-bin-hadoop2.7.tgz’

spark-3.0.3-bin-had 100%[===================>] 210.19M   162MB/s    in 1.3s    

2021-11-28 04:27:22 (162 MB/s) - ‘spark-3.0.3-bin-hadoop2.7.tgz’ saved [220400553/220400553]



In [ ]:
!tar xf /content/spark-3.0.3-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init("/content/spark-3.0.3-bin-hadoop2.7")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
data.iloc[:,3:] = data.iloc[:,3:].astype(str).astype(float)
data['radiant_win'] = data['radiant_win']*1
data = data.fillna(0)

In [ ]:
spark_data = spark.createDataFrame(data)
spark_data.printSchema()
spark_data.show()

root
 |-- match_id: long (nullable = true)
 |-- duration: long (nullable = true)
 |-- radiant_win: long (nullable = true)
 |-- radiant_gold_adv_at_0: double (nullable = true)
 |-- radiant_gold_adv_at_1: double (nullable = true)
 |-- radiant_gold_adv_at_2: double (nullable = true)
 |-- radiant_gold_adv_at_3: double (nullable = true)
 |-- radiant_gold_adv_at_4: double (nullable = true)
 |-- radiant_gold_adv_at_5: double (nullable = true)
 |-- radiant_gold_adv_at_6: double (nullable = true)
 |-- radiant_gold_adv_at_7: double (nullable = true)
 |-- radiant_gold_adv_at_8: double (nullable = true)
 |-- radiant_gold_adv_at_9: double (nullable = true)
 |-- radiant_gold_adv_at_10: double (nullable = true)
 |-- radiant_gold_adv_at_11: double (nullable = true)
 |-- radiant_gold_adv_at_12: double (nullable = true)
 |-- radiant_gold_adv_at_13: double (nullable = true)
 |-- radiant_gold_adv_at_14: double (nullable = true)
 |-- radiant_gold_adv_at_15: double (nullable = true)
 |-- radiant_gold_adv_at

In [ ]:
spark_data.select('duration').describe().show()

+-------+-----------------+
|summary|         duration|
+-------+-----------------+
|  count|             2747|
|   mean|2245.389879868948|
| stddev| 624.035619395898|
|    min|              615|
|    max|             4787|
+-------+-----------------+



In [ ]:
for i in spark_data.columns:
  if not(isinstance(spark_data.select(i).take(1)[0][0], str)):
    print('Correlation to Radiant Victory for', i , spark_data.stat.corr('radiant_win', i))

Correlation to Radiant Victory for match_id 0.039469802755397035
Correlation to Radiant Victory for duration -0.03897570218250891
Correlation to Radiant Victory for radiant_win 1.0
Correlation to Radiant Victory for radiant_gold_adv_at_0 nan
Correlation to Radiant Victory for radiant_gold_adv_at_1 0.08535821303005801
Correlation to Radiant Victory for radiant_gold_adv_at_2 0.08424217174865778
Correlation to Radiant Victory for radiant_gold_adv_at_3 0.09822052885104753
Correlation to Radiant Victory for radiant_gold_adv_at_4 0.12255398543207725
Correlation to Radiant Victory for radiant_gold_adv_at_5 0.14751305830601513
Correlation to Radiant Victory for radiant_gold_adv_at_6 0.17086838405078794
Correlation to Radiant Victory for radiant_gold_adv_at_7 0.1881252482901254
Correlation to Radiant Victory for radiant_gold_adv_at_8 0.20204908314114242
Correlation to Radiant Victory for radiant_gold_adv_at_9 0.2171957200204668
Correlation to Radiant Victory for radiant_gold_adv_at_10 0.2356731

In [ ]:
from pyspark.ml.linalg import Vectors
def transformToLabeledPoint(row):
  lp = (row['radiant_win'], Vectors.dense([row['radiant_gold_adv_at_39'], row['radiant_xp_adv_at_39']])) #
  return lp

sparkDataLp = spark_data.map(transformToLabeledPoint)
autoDF = spark.createDataFrame(autoLp, ['label','features'])
autoDF.select('label','features').show(10)

# Pandas Experimentation


In [12]:
from sklearn.preprocessing import StandardScaler

In [14]:
%%time
normalized_data = data.copy()
normalized_data.iloc[:,3:] = StandardScaler().fit_transform(normalized_data.iloc[:,3:])

normalized_data_target = normalized_data['radiant_win']
normalized_data.drop('radiant_win', axis=1, inplace=True)

CPU times: user 1.65 s, sys: 14.7 ms, total: 1.66 s
Wall time: 1.67 s


In [ ]:
normalized_data

,match_id,duration,radiant_gold_adv_at_0,radiant_gold_adv_at_1,radiant_gold_adv_at_2,radiant_gold_adv_at_3,radiant_gold_adv_at_4,radiant_gold_adv_at_5,radiant_gold_adv_at_6,radiant_gold_adv_at_7,radiant_gold_adv_at_8,radiant_gold_adv_at_9,radiant_gold_adv_at_10,radiant_gold_adv_at_11,radiant_gold_adv_at_12,radiant_gold_adv_at_13,radiant_gold_adv_at_14,radiant_gold_adv_at_15,radiant_gold_adv_at_16,radiant_gold_adv_at_17,radiant_gold_adv_at_18,radiant_gold_adv_at_19,radiant_gold_adv_at_20,radiant_gold_adv_at_21,radiant_gold_adv_at_22,radiant_gold_adv_at_23,radiant_gold_adv_at_24,radiant_gold_adv_at_25,radiant_gold_adv_at_26,radiant_gold_adv_at_27,radiant_gold_adv_at_28,radiant_gold_adv_at_29,radiant_gold_adv_at_30,radiant_gold_adv_at_31,radiant_gold_adv_at_32,radiant_gold_adv_at_33,radiant_gold_adv_at_34,radiant_gold_adv_at_35,radiant_gold_adv_at_36,radiant_gold_adv_at_37,...,radiant_xp_adv_at_40,radiant_xp_adv_at_41,radiant_xp_adv_at_42,radiant_xp_adv_at_43,radiant_xp_adv_at_44,radiant_xp_adv_at_45,radiant_xp_adv_at_46,radiant_xp_adv_at_47,radiant_xp_adv_at_48,radiant_xp_adv_at_49,radiant_xp_adv_at_50,radiant_xp_adv_at_51,radiant_xp_adv_at_52,radiant_xp_adv_at_53,radiant_xp_adv_at_54,radiant_xp_adv_at_55,radiant_xp_adv_at_56,radiant_xp_adv_at_57,radiant_xp_adv_at_58,radiant_xp_adv_at_59,radiant_xp_adv_at_60,radiant_xp_adv_at_61,radiant_xp_adv_at_62,radiant_xp_adv_at_63,radiant_xp_adv_at_64,radiant_xp_adv_at_65,radiant_xp_adv_at_66,radiant_xp_adv_at_67,radiant_xp_adv_at_68,radiant_xp_adv_at_69,radiant_xp_adv_at_70,radiant_xp_adv_at_71,radiant_xp_adv_at_72,radiant_xp_adv_at_73,radiant_xp_adv_at_74,radiant_xp_adv_at_75,radiant_xp_adv_at_76,radiant_xp_adv_at_77,radiant_xp_adv_at_78,radiant_xp_adv_at_79
2,6292239339,1202,0.038488,0.021664,0.022833,0.035075,0.030119,-0.338320,-0.634363,-0.870276,-1.019208,-0.818333,-0.648939,-0.504316,-0.369303,-0.196500,-0.053801,0.061872,0.120548,0.020766,-0.068053,-0.146014,-0.211727,-0.270166,-0.320903,-0.364085,-0.394117,-0.411432,-0.425033,-0.437684,-0.429102,-0.382812,-0.337233,-0.297662,-0.265736,-0.240424,-0.214930,-0.190412,-0.160711,-0.128847,...,0.949598,0.933289,0.906011,0.875742,0.849361,0.824740,0.799144,0.775590,0.818354,0.889650,0.947286,1.000594,1.061120,1.115206,1.163874,1.206826,1.237681,1.264991,1.291955,1.314705,1.432530,1.572792,1.701166,1.820736,1.747651,1.632532,1.523377,1.423669,1.374079,1.333327,1.292636,1.253019,1.423658,1.601134,1.761212,1.896369,1.997248,2.061923,2.072002,2.054767
3,6292239088,1275,0.038488,-0.072185,-0.089527,-0.100971,-0.041816,0.193303,0.377373,0.525131,0.368503,-0.030480,-0.361885,-0.637156,-0.863027,-1.036613,-1.180600,-1.304480,-1.304339,-1.297295,-1.285296,-1.267989,-1.209859,-1.152684,-1.102993,-1.158034,-1.328016,-1.480928,-1.614889,-1.627505,-1.586442,-1.550384,-1.513407,-1.410991,-1.310351,-1.213817,-1.159116,-1.329850,-1.481034,-1.623815,...,-0.293874,-0.173844,-0.215952,-0.340267,-0.453624,-0.557780,-0.474564,-0.365939,-0.263318,-0.178610,-0.151171,-0.122612,-0.090946,-0.150771,-0.375710,-0.580927,-0.775422,-0.800535,-0.707950,-0.618836,-0.534550,-0.701938,-0.907961,-1.100444,-1.273812,-1.307814,-1.338558,-1.372359,-1.420328,-1.496138,-1.565062,-1.629287,-1.743434,-1.891807,-2.019880,-2.125836,-2.195886,-2.234121,-2.219527,-2.178004
4,6292238931,1183,0.038488,1.102728,1.317135,1.602218,1.872939,1.654940,1.398216,1.183713,0.995368,0.837627,0.703335,0.585011,0.476874,0.379343,0.295361,0.220340,0.157289,0.105298,0.060337,0.020529,-0.010896,-0.032837,-0.038466,-0.041518,-0.042219,-0.042412,0.001056,0.040911,0.073398,0.100567,0.220252,0.335962,0.440568,0.535395,0.438013,0.288890,0.151232,0.022560,...,-0.328579,-0.406616,-0.463090,-0.488451,-0.511692,-0.533176,-0.524595,-0.426572,-0.334045,-0.253172,-0.171808,-0.042864,0.081488,0.194539,0.297936,0.142567,-0.018596,-0.169882,-0.310119,-0.230211,-0.100500,0.020605,0.137126,0.345160,0.595620,0.828978,1.046713,1.122155,1.050791,0.982552,0.916604,0.867277,0.846363,0.824161,0.801901,0.808271,0.972699,1.

In [ ]:
normalized_data_target

2        True
3       False
4        True
5        True
6       False
        ...  
2747    False
2748     True
2749    False
2750    False
2751    False
Name: radiant_win, Length: 2747, dtype: bool

In [16]:
%%time
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(normalized_data, normalized_data_target, test_size = 0.2, random_state = 42)

CPU times: user 6.12 ms, sys: 0 ns, total: 6.12 ms
Wall time: 12.5 ms


In [17]:
%%time
from sklearn import tree 

tree_clf = tree.DecisionTreeClassifier()
tree_clf = tree_clf.fit(x_train, y_train)

CPU times: user 340 ms, sys: 12.9 ms, total: 353 ms
Wall time: 552 ms


In [ ]:
import graphviz 
dot_data = tree.export_graphviz(tree_clf, out_file=None) 
graph = graphviz.Source(dot_data) 

In [18]:
%%time
from sklearn import metrics

y_pred = tree_clf.predict(x_test)
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9509090909090909
CPU times: user 5.32 ms, sys: 0 ns, total: 5.32 ms
Wall time: 5.33 ms


Let's try to predict time now

In [19]:
normalized_data2 = normalized_data.copy()

normalized_data2_target = normalized_data['duration']
normalized_data2.drop('duration', axis=1, inplace=True)
normalized_data2.drop('match_id', axis=1, inplace=True)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(normalized_data2, normalized_data2_target, test_size = 0.2, random_state = 42)

In [115]:
@ray.remote
def tree_for_time(tree_clf):
  tree_clf = tree_clf.fit(x_train, y_train) 
  return tree_clf

In [21]:
%%time
tree_clf = tree.DecisionTreeClassifier()
tree_clf = tree_clf.fit(x_train, y_train)

CPU times: user 36.5 s, sys: 102 ms, total: 36.6 s
Wall time: 36.4 s


In [117]:
%%time
tree_ray_clf = tree.DecisionTreeClassifier()
ref = tree_for_time.remote(tree_ray_clf)
tree_ray_clf = ray.get(ref)

CPU times: user 1 s, sys: 137 ms, total: 1.14 s
Wall time: 42.8 s


In [ ]:
dot_data = tree.export_graphviz(tree_clf, out_file=None) 
graph = graphviz.Source(dot_data) 

In [114]:
%%time
y_pred = tree_ray_clf.predict(x_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

NotFittedError: ignored

And now with something that is actually used for predicting time

In [23]:
%%time
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha = 0.01)
ridge_reg.fit(x_train, y_train)

CPU times: user 15.6 ms, sys: 28.4 ms, total: 43.9 ms
Wall time: 40.8 ms


In [24]:
%%time
y_pred = ridge_reg.predict(x_test)
print('Accuracy:', metrics.mean_absolute_error(y_test, y_pred))

Accuracy: 502.1913638277841
CPU times: user 6.56 ms, sys: 7.07 ms, total: 13.6 ms
Wall time: 11.6 ms


In [25]:
%%time
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2, kernel = 'rbf'))
regr.fit(x_train, y_train)

CPU times: user 518 ms, sys: 11.7 ms, total: 530 ms
Wall time: 532 ms


In [26]:
%%time
y_pred = regr.predict(x_test)
print('Error Percentage:', metrics.mean_absolute_percentage_error(y_test, y_pred) * 100, '%')

Error Percentage: 23.88776090093024 %
CPU times: user 188 ms, sys: 817 µs, total: 189 ms
Wall time: 198 ms
